# Model recommendation with lighfm

### Import libraries

In [1]:
import os
import sys
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from lightfm import cross_validation
import scipy.sparse as sp
from scipy import sparse

### Defining variables

In [2]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [3]:
dtype_df_valid = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"timestampHistory" : 'string'
}

dtype_df_train_score = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"score" : 'Float32'
}

In [4]:
K_LIGHTFM_ITEMS = 6
K_POPULAR_ITEMS = 6
K_SAMPLED_ITEMS = 3
LIMIT_N_ROWS = 1000
N_PARTS_DASK = 3
SEED = 42

### Retrieve data

In [5]:
import pandas as pd
# path config

# df_valid = pd.read_csv(config["VALID_DF"],dtype=dtype_df_valid)
df_valid = pd.read_csv(config["VALID_DF"],dtype=dtype_df_valid,nrows=LIMIT_N_ROWS)
df_valid.dropna()

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]
2,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,['857aa90f-a7ec-410d-ba82-dfa4f85d4e71'],[1660561649242]
3,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,['b7b90e18-7613-4ca0-a8fc-fd69addfcd85' '835f...,[1660533830245 1660540831707 1660542659111 166...
4,a120515626fe5d12b22b7d5a7c5008912cc69284aa26cc...,Logged,['9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6' 'b8eb...,[1660548813953 1660572329731 1660594848200]
...,...,...,...,...
995,0bca64a686750abbb1ba077318fb7d3b094f50c55374a5...,Logged,['a89983c3-e220-4a5f-ab23-5edd8b2b914c'],[1660646489438]
996,71925c3e31da91c811d2b445d169cdf737889ccd02e7e3...,Logged,['cf7c4c6c-38e5-43bb-ad54-c33ad7253ac2'],[1660591462479]
997,0fee67e553a97ea13828b0ae5f3a9d6450418f19c05c47...,Logged,['6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3'],[1660533586069]
998,9cad524b959bbbcd00995bc25fbea95dedcce71dd7bee9...,Logged,['6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3' 'a41b...,[1660558252865 1660702436161]


In [6]:
df_news = pd.read_csv(config["DF_ITEMS_FEATURE"])
df_news.drop(columns=["Unnamed: 0"],inplace=True)
df_news.head(3)

,page,age_exp_normalized,ageCategories
0,7371a9b5-5824-4c57-8704-00a74feebe79,0.151439,very-old
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,0.140788,very-old
2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,0.122610,very-old


### Make predictions to known and unknowm on same recommendation function with pkls

In [7]:
from utils.custom_treat_data_funcs import explode_df_columns

cols_to_explode = ["history", "timestampHistory"]
cols_and_id = cols_to_explode.copy()
cols_and_id.insert(0,"userId")
cols_and_id.append("userType")
cols_and_id = tuple(cols_and_id)
exploded_df = explode_df_columns(df_valid.loc[:,cols_and_id], cols_to_explode)

In [8]:
df_valid

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]
2,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,['857aa90f-a7ec-410d-ba82-dfa4f85d4e71'],[1660561649242]
3,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,['b7b90e18-7613-4ca0-a8fc-fd69addfcd85' '835f...,[1660533830245 1660540831707 1660542659111 166...
4,a120515626fe5d12b22b7d5a7c5008912cc69284aa26cc...,Logged,['9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6' 'b8eb...,[1660548813953 1660572329731 1660594848200]
...,...,...,...,...
995,0bca64a686750abbb1ba077318fb7d3b094f50c55374a5...,Logged,['a89983c3-e220-4a5f-ab23-5edd8b2b914c'],[1660646489438]
996,71925c3e31da91c811d2b445d169cdf737889ccd02e7e3...,Logged,['cf7c4c6c-38e5-43bb-ad54-c33ad7253ac2'],[1660591462479]
997,0fee67e553a97ea13828b0ae5f3a9d6450418f19c05c47...,Logged,['6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3'],[1660533586069]
998,9cad524b959bbbcd00995bc25fbea95dedcce71dd7bee9...,Logged,['6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3' 'a41b...,[1660558252865 1660702436161]


In [9]:
exploded_df

,userId,history,timestampHistory,userType
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,be89a7da-d9fa-49d4-9fdc-388c27a15bc8,1660533136590,Logged
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,01c59ff6-fb82-4258-918f-2910cb2d4c52,1660672113513,Logged
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,77901133-aee7-4f7b-afc0-652231d76fe9,1660556860253,Logged
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,857aa90f-a7ec-410d-ba82-dfa4f85d4e71,1660561649242,Logged
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,b7b90e18-7613-4ca0-a8fc-fd69addfcd85,1660533830245,Logged
...,...,...,...,...
2287,71925c3e31da91c811d2b445d169cdf737889ccd02e7e3...,cf7c4c6c-38e5-43bb-ad54-c33ad7253ac2,1660591462479,Logged
2288,0fee67e553a97ea13828b0ae5f3a9d6450418f19c05c47...,6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3,1660533586069,Logged
2289,9cad524b959bbbcd00995bc25fbea95dedcce71dd7bee9...,6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3,1660558252865,Logged
2290,9cad524b959bbbcd00995bc25fbea95dedcce71dd7bee9...,a41bed7d-5376-479f-b503-a8b046a4c7e3,1660702436161,Logged


In [10]:
exploded_df["check_hifens"] = exploded_df["history"].str.split('-').apply(lambda x : len(x) == 5).astype("bool")
exploded_df["check_size"] = exploded_df["history"].str.replace(r'-', '', regex=True).apply(lambda x : len(x) == 32).astype("bool")
exploded_df["check_chars"] = exploded_df["history"].str.replace(r'-', '', regex=True).str.replace(r'[a-f0-9]', '', regex=True).apply(lambda x : x == '').astype("bool")
exploded_df["check_history"] = (exploded_df["check_hifens"] & exploded_df["check_size"] & exploded_df["check_chars"]).astype("bool")
exploded_df["check_history"].value_counts()

check_history
True     2207
False      85
Name: count, dtype: int64

In [11]:
exploded_df_valid = exploded_df[~exploded_df["check_history"]==False][["userId","userType","history"]]
exploded_df_valid

,userId,userType,history
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,be89a7da-d9fa-49d4-9fdc-388c27a15bc8
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,01c59ff6-fb82-4258-918f-2910cb2d4c52
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,77901133-aee7-4f7b-afc0-652231d76fe9
3,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,857aa90f-a7ec-410d-ba82-dfa4f85d4e71
4,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,b7b90e18-7613-4ca0-a8fc-fd69addfcd85
...,...,...,...
2287,71925c3e31da91c811d2b445d169cdf737889ccd02e7e3...,Logged,cf7c4c6c-38e5-43bb-ad54-c33ad7253ac2
2288,0fee67e553a97ea13828b0ae5f3a9d6450418f19c05c47...,Logged,6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3
2289,9cad524b959bbbcd00995bc25fbea95dedcce71dd7bee9...,Logged,6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3
2290,9cad524b959bbbcd00995bc25fbea95dedcce71dd7bee9...,Logged,a41bed7d-5376-479f-b503-a8b046a4c7e3


In [12]:
df_valid = exploded_df_valid.groupby(["userId","userType"]).history.unique().to_frame().reset_index()
df_valid

/tmp/ipykernel_399199/978289096.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_valid = exploded_df_valid.groupby(["userId","userType"]).history.unique().to_frame().reset_index()


,userId,userType,history
0,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,Logged,"[d730c4a6-e8f6-4fde-b73a-afbe148479cd, be89a7d..."
1,00aee36362731c3a7c6891e82680e98400f0feea68e9b9...,Logged,"[d730c4a6-e8f6-4fde-b73a-afbe148479cd, 109adb0..."
2,00bcf7e66456386d571b01ce096bd50379d16c7e3c1e01...,Logged,[72384013-5d8a-489f-ae0e-53cc739e4f36]
3,00c35ef7c5a6c85ab17202687cd0e1c49b852d944ad11b...,Logged,[4c3d47a1-6f4b-424f-8944-6c227e686c5c]
4,00c4a6454163650d3424a18bdeb824df669d009e68b8e0...,Logged,[5bf8b3a3-3916-40ca-8672-a9f3df311825]
...,...,...,...
974,fdc023689a8f9eb312e7b03ea1c18bae5d319e3de74d73...,Logged,[433dbb85-cc21-4624-a74b-3db9d34e0e4f]
975,fdd2d7a7d9f8a10833467662863b5becd9db2447f64e84...,Logged,[6d5452ec-5eec-4a57-b9bf-a00530176d31]
976,fe020b3c69c19db98ecd74b1c04cdc0e12993ca280a855...,Logged,"[a60b3e2f-c009-4a8b-9bcd-068e7d3c927a, 1b4dd5f..."
977,ffb3c1cb04ed5a54f49ef35dc5cd1e482d689d2020408b...,Logged,[44b83851-8a5a-49fa-9b6d-66ca16952cde]


In [13]:
import random

random.seed(SEED)

In [14]:
import pickle
from utils.custom_data_structs import UserItemData

loaded_model = pickle.load(open('artifacts/lightfm_model.pkl', 'rb'))
loaded_user_item_data = pickle.load(open('artifacts/user_item_data.pkl', 'rb'))

loaded_n_users, loaded_n_items = loaded_user_item_data.interactions_shape

In [15]:
def format_newuser_input(user_feature_map, user_feature_list):
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        print("new user feature encountered '{}'".format(feature))
        pass
  #print("target indices: {}".format(target_indices))
  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

In [16]:
def get_scores_from_model(user_hash:str,user_feature_list:list|str,user_item_data:UserItemData,model:LightFM):
    """
    This function 
    """
    if isinstance(user_feature_list, str):
        user_feature_list = [user_feature_list]
    try:
        user_x = user_item_data.user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(loaded_n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_item_data.user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(loaded_n_items), user_features=new_user_features)
    return scores

In [17]:
def get_top_items_by_model_scores(scores, user_item_data):
    """
    This function verifies if the users is known or new, and makes recommendations depending on this verification.
    The top K_LIGHTFM_ITEMS recommendations from the list are returned.
    """
    
    top_k_indices = np.argsort(-scores)[:K_LIGHTFM_ITEMS]  # Sort scores in descending order and take the top K_ITEMS
    return [user_item_data.item_id_map_reverse[i] for i in top_k_indices]

In [18]:
def recommend_by_model_scores(user_hash:str,user_feature_list:list|str,user_item_data:UserItemData,model:LightFM):
    """
    This function 
    """
    
    scores = get_scores_from_model(user_hash,user_feature_list,user_item_data,model)
    return get_top_items_by_model_scores(scores, user_item_data)

In [19]:
def count_histories_by_popularity(df:pd.DataFrame):
    """
    This function 
    """
    
    return df.groupby(["history"]).size().sort_values(ascending=False).astype(dtype="UInt16")

In [20]:
def get_df_most_popular_histories(df_count_histories:pd.DataFrame):
    return df_count_histories.iloc[:K_POPULAR_ITEMS]

In [21]:
def get_df_most_unpopular_histories(df_count_histories:pd.DataFrame):
    return df_count_histories.iloc[K_POPULAR_ITEMS:]

In [22]:
def recommend_by_most_popular(df:pd.DataFrame):
    """
    This function 
    """
    df_count_histories = count_histories_by_popularity(df)
    df_popular_stories = get_df_most_popular_histories(df_count_histories)
    return list(set(df_popular_stories.keys()))

In [23]:
def recommend_by_weighted_random(df_unpopular_histories:pd.DataFrame, unpopular_weights:list):
    """
    This function 
    """
    random_k_histories = df_unpopular_histories.sample(n=K_SAMPLED_ITEMS, weights=unpopular_weights, random_state=random.randint(0, 50))
    # return random_k_histories
    return list(set(random_k_histories.keys()))

In [24]:
def order_df_by_most_recent(df:pd.DataFrame, history_list:list):
    return df[df["history"].isin(history_list)].sort_values("age_exp_normalized",ascending=False)

In [25]:
def check_valid_recommendations(validation_history_hash, recommeded_histories):
    """
    Check if validation_history_hash is in recommeded_histories
    """
    recommendation_in_validation_flag = False
    if validation_history_hash in recommeded_histories:
        recommendation_in_validation_flag = True
    return recommendation_in_validation_flag
    

In [26]:
def list_intersection(list1:list, list2:list):
    return list(set(list1) & set(list2))

In [27]:
top_k_popular = recommend_by_most_popular(exploded_df_valid)
print(top_k_popular)

['82e7b237-e7ea-4cf5-b44c-55385e2cdbb0', 'eb23272d-8e6c-479d-b972-eabeb5f6f3dd', '9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6', '4c3d47a1-6f4b-424f-8944-6c227e686c5c', 'be89a7da-d9fa-49d4-9fdc-388c27a15bc8', 'aeab0e46-f1e4-41e9-821b-571255c41f69']


In [28]:
df_count_histories = count_histories_by_popularity(exploded_df_valid)
df_unpopular_histories = get_df_most_unpopular_histories(df_count_histories)
unpopular_weights = tuple(df_unpopular_histories.values.tolist())

top_k_randomw = recommend_by_weighted_random(df_unpopular_histories, unpopular_weights)
print(top_k_randomw)

['b1e41505-8fd5-40ea-9363-b45ee48aa556', '72384013-5d8a-489f-ae0e-53cc739e4f36', '27d9047e-e42a-46e1-862b-6089b1707d6f']


In [29]:
import dask.dataframe as dd

In [30]:
ddf_valid = dd.from_pandas(df_valid, npartitions=N_PARTS_DASK)

df_valid["recommendedByModel"] = ddf_valid.apply(lambda x :
    recommend_by_model_scores(
        x.userId,
        x.userType,
        loaded_user_item_data,
        loaded_model),
        axis=1,
        meta=('string','string')).compute()


In [31]:
ddf_valid = dd.from_pandas(df_valid, npartitions=N_PARTS_DASK)

df_valid["recommendedByRandomW"] = ddf_valid.apply(lambda x :
    recommend_by_weighted_random(df_unpopular_histories, unpopular_weights),
    axis=1, meta=('object','object')).compute()


In [32]:
from utils.custom_treat_data_funcs import transform_text_to_list

df_valid["recommendedByPopular"] = str(top_k_popular)
df_valid["recommendedByPopular"] = df_valid["recommendedByPopular"].apply(transform_text_to_list)


In [33]:
df_valid["recommendedMerged"] = df_valid["recommendedByRandomW"] + df_valid["recommendedByPopular"] + df_valid["recommendedByModel"]

In [34]:
df_valid.head(2)

,userId,userType,history,recommendedByModel,recommendedByRandomW,recommendedByPopular,recommendedMerged
0,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,Logged,"[d730c4a6-e8f6-4fde-b73a-afbe148479cd, be89a7d...","[78fe8ccc-99cf-4da8-88d1-080a13ab6ca4, 870c8a9...","[444db2d3-2856-4307-8a36-09b15651fb8b, 6bd12ed...","[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...","[444db2d3-2856-4307-8a36-09b15651fb8b, 6bd12ed..."
1,00aee36362731c3a7c6891e82680e98400f0feea68e9b9...,Logged,"[d730c4a6-e8f6-4fde-b73a-afbe148479cd, 109adb0...","[78fe8ccc-99cf-4da8-88d1-080a13ab6ca4, 870c8a9...","[f2f45310-e13d-4deb-acb9-170ffc330335, a07ab9d...","[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...","[f2f45310-e13d-4deb-acb9-170ffc330335, a07ab9d..."


In [35]:
df_valid.drop(columns=["recommendedByModel","recommendedByRandomW","recommendedByPopular"],inplace=True)
df_valid_recommends = df_valid.drop(columns=["userType","history"])

In [36]:
df_valid_recommends

,userId,recommendedMerged
0,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,"[444db2d3-2856-4307-8a36-09b15651fb8b, 6bd12ed..."
1,00aee36362731c3a7c6891e82680e98400f0feea68e9b9...,"[f2f45310-e13d-4deb-acb9-170ffc330335, a07ab9d..."
2,00bcf7e66456386d571b01ce096bd50379d16c7e3c1e01...,"[3039fc3f-966c-4067-8ea9-33011ddee5d7, bb4aa15..."
3,00c35ef7c5a6c85ab17202687cd0e1c49b852d944ad11b...,"[d0d32897-99b9-4a4d-9ed3-debd731a3821, 54915fb..."
4,00c4a6454163650d3424a18bdeb824df669d009e68b8e0...,"[9c61f592-51b7-4cb2-bc52-684253ec2a2b, 290c3a5..."
...,...,...
974,fdc023689a8f9eb312e7b03ea1c18bae5d319e3de74d73...,"[9c61f592-51b7-4cb2-bc52-684253ec2a2b, 290c3a5..."
975,fdd2d7a7d9f8a10833467662863b5becd9db2447f64e84...,"[64406c3a-aa01-42a6-af5a-5843db3d5f33, 0bf174c..."
976,fe020b3c69c19db98ecd74b1c04cdc0e12993ca280a855...,"[d730c4a6-e8f6-4fde-b73a-afbe148479cd, 2c6383b..."
977,ffb3c1cb04ed5a54f49ef35dc5cd1e482d689d2020408b...,"[ed82a14e-ae58-477a-b12f-0e16c114a095, ac820b4..."


In [37]:
from utils.custom_treat_data_funcs import explode_df_columns

cols_to_explode = ["recommendedMerged"]
cols_and_id = ["userId"]
cols_and_id = tuple(cols_and_id+cols_to_explode)
print(cols_and_id)
df_valid_recommends_exploded = explode_df_columns(df_valid_recommends.loc[:,cols_and_id], cols_to_explode)

('userId', 'recommendedMerged')


In [38]:
df_valid_recommends_exploded

,userId,recommendedMerged
0,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,444db2d3-2856-4307-8a36-09b15651fb8b
1,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,6bd12ed5-de5a-4d7a-aa0d-ba00096aa181
2,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,27112b95-15dc-4363-9d7a-0baf0d23b206
3,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,82e7b237-e7ea-4cf5-b44c-55385e2cdbb0
4,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,eb23272d-8e6c-479d-b972-eabeb5f6f3dd
...,...,...
14680,fff46e72c87ef6d8e149b0a60f3346a84256b2d30c04bc...,78fe8ccc-99cf-4da8-88d1-080a13ab6ca4
14681,fff46e72c87ef6d8e149b0a60f3346a84256b2d30c04bc...,50392ead-f267-4b08-ab03-7f7c4440a8b5
14682,fff46e72c87ef6d8e149b0a60f3346a84256b2d30c04bc...,baba0fcf-80b5-4c39-90c1-bde6bd56b72f
14683,fff46e72c87ef6d8e149b0a60f3346a84256b2d30c04bc...,9ae52dae-de91-4969-b4ea-c85a1e6f7491


In [39]:
df_valid_exploded_rec_age = pd.merge(df_valid_recommends_exploded, df_news, left_on='recommendedMerged', right_on='page', how='left')
df_valid_exploded_rec_age.drop(columns=["page","ageCategories"],inplace=True) # 'page' is the same as 'history', 'ageCategories' will not be used, only numeric
df_valid_exploded_rec_age[~df_valid_exploded_rec_age["age_exp_normalized"].notna()]

,userId,recommendedMerged,age_exp_normalized


In [40]:
df_valid_exploded_rec_age_top_recent = df_valid_exploded_rec_age.sort_values("age_exp_normalized",ascending=False).groupby("userId").head(5).sort_index()
df_valid_exploded_rec_age_top_recent

,userId,recommendedMerged,age_exp_normalized
2,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,27112b95-15dc-4363-9d7a-0baf0d23b206,1.0
3,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,82e7b237-e7ea-4cf5-b44c-55385e2cdbb0,1.0
5,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6,1.0
6,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,4c3d47a1-6f4b-424f-8944-6c227e686c5c,1.0
7,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,be89a7da-d9fa-49d4-9fdc-388c27a15bc8,1.0
...,...,...,...
14673,fff46e72c87ef6d8e149b0a60f3346a84256b2d30c04bc...,82e7b237-e7ea-4cf5-b44c-55385e2cdbb0,1.0
14674,fff46e72c87ef6d8e149b0a60f3346a84256b2d30c04bc...,eb23272d-8e6c-479d-b972-eabeb5f6f3dd,1.0
14675,fff46e72c87ef6d8e149b0a60f3346a84256b2d30c04bc...,9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6,1.0
14676,fff46e72c87ef6d8e149b0a60f3346a84256b2d30c04bc...,4c3d47a1-6f4b-424f-8944-6c227e686c5c,1.0


In [41]:
df_final_recommends_exploded_top_recent = df_valid_exploded_rec_age_top_recent.drop(columns="age_exp_normalized")
df_final_recommends_exploded_top_recent.head(2)

,userId,recommendedMerged
2,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,27112b95-15dc-4363-9d7a-0baf0d23b206
3,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,82e7b237-e7ea-4cf5-b44c-55385e2cdbb0


In [42]:
df_final_recommends = df_final_recommends_exploded_top_recent.groupby("userId").recommendedMerged.unique()

In [43]:
df_final_recommends = pd.merge(df_final_recommends, df_valid[["userId","history"]], left_on='userId', right_on='userId', how='left')
df_final_recommends

,userId,recommendedMerged,history
0,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,"[27112b95-15dc-4363-9d7a-0baf0d23b206, 82e7b23...","[d730c4a6-e8f6-4fde-b73a-afbe148479cd, be89a7d..."
1,00aee36362731c3a7c6891e82680e98400f0feea68e9b9...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...","[d730c4a6-e8f6-4fde-b73a-afbe148479cd, 109adb0..."
2,00bcf7e66456386d571b01ce096bd50379d16c7e3c1e01...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...",[72384013-5d8a-489f-ae0e-53cc739e4f36]
3,00c35ef7c5a6c85ab17202687cd0e1c49b852d944ad11b...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...",[4c3d47a1-6f4b-424f-8944-6c227e686c5c]
4,00c4a6454163650d3424a18bdeb824df669d009e68b8e0...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...",[5bf8b3a3-3916-40ca-8672-a9f3df311825]
...,...,...,...
974,fdc023689a8f9eb312e7b03ea1c18bae5d319e3de74d73...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...",[433dbb85-cc21-4624-a74b-3db9d34e0e4f]
975,fdd2d7a7d9f8a10833467662863b5becd9db2447f64e84...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...",[6d5452ec-5eec-4a57-b9bf-a00530176d31]
976,fe020b3c69c19db98ecd74b1c04cdc0e12993ca280a855...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...","[a60b3e2f-c009-4a8b-9bcd-068e7d3c927a, 1b4dd5f..."
977,ffb3c1cb04ed5a54f49ef35dc5cd1e482d689d2020408b...,"[ac820b47-0f63-44c3-a431-85b1995c3724, 7b470f4...",[44b83851-8a5a-49fa-9b6d-66ca16952cde]


In [44]:
df_final_recommends["matched_recommendations"] = df_final_recommends.apply(lambda x :
    list_intersection(x.history, x.recommendedMerged),
    axis=1
)

In [45]:
df_final_recommends["count_matches"] = df_final_recommends["matched_recommendations"].apply(lambda x : len(x))

In [46]:
df_final_recommends[df_final_recommends["count_matches"]>0]

,userId,recommendedMerged,history,matched_recommendations,count_matches
0,0067175c1d42a35553b857d4d67f6d950598455f0f43de...,"[27112b95-15dc-4363-9d7a-0baf0d23b206, 82e7b23...","[d730c4a6-e8f6-4fde-b73a-afbe148479cd, be89a7d...",[be89a7da-d9fa-49d4-9fdc-388c27a15bc8],1
1,00aee36362731c3a7c6891e82680e98400f0feea68e9b9...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...","[d730c4a6-e8f6-4fde-b73a-afbe148479cd, 109adb0...",[be89a7da-d9fa-49d4-9fdc-388c27a15bc8],1
3,00c35ef7c5a6c85ab17202687cd0e1c49b852d944ad11b...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...",[4c3d47a1-6f4b-424f-8944-6c227e686c5c],[4c3d47a1-6f4b-424f-8944-6c227e686c5c],1
13,0407f0fa3f07c6228c313bd764c51e5b2b47b66d630767...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...","[4c3d47a1-6f4b-424f-8944-6c227e686c5c, 426e977...",[4c3d47a1-6f4b-424f-8944-6c227e686c5c],1
19,05cf81e7acee993069a54586a5f07224d5bbbe3e51e3fe...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...",[9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6],[9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6],1
...,...,...,...,...,...
961,f94f2d4c131d2240f7b9c5e3162f9ac6afa16c975a4663...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...","[82206221-05a0-4f89-aa10-e8c8dc0cc418, 4c3d47a...",[4c3d47a1-6f4b-424f-8944-6c227e686c5c],1
963,fa639a5d4a68ee851643301355a07aeceaaa2817d80cfd...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...","[4c3d47a1-6f4b-424f-8944-6c227e686c5c, fe31a3a...","[4c3d47a1-6f4b-424f-8944-6c227e686c5c, eb23272...",2
968,fbbced0dca062557d110e57920718858d83ced42045255...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...","[4c3d47a1-6f4b-424f-8944-6c227e686c5c, fd2e3f2...",[4c3d47a1-6f4b-424f-8944-6c227e686c5c],1
970,fd15d8f89c7bfb415d408bca770fe0e27d78ee9a641fb2...,"[82e7b237-e7ea-4cf5-b44c-55385e2cdbb0, eb23272...","[9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6, 1266a9a...",[9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6],1


In [47]:
df_final_recommends["count_matches"].sum()

np.int64(237)